<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200">
</div>

# Hybrid Compute for QEC workflows on Quantinuum H-Series

Quantinuum has released a new H-Series capability, hybrid compute for Quantum Error Correction (QEC) workflows. This new capability executes `WebAssembly` (`Wasm`) in the H-Series stack and enables classical computation during real-time execution of a quantum circuit. Unlocking experimentation of basic QEC workflows is an important milestone, as Quantinuum plans to scale its H-Series hardware and deploy practical QEC schemes at scale.

**Note:** This notebook uses existing Wasm examples provided in this folder's subdirectories. To create and run your own `Wasm` functions, you'll need to set up an environment on your machine to support this workflow. Instructions for how to do this are given in this folder's README.


Note the `Wasm` module imported from `pytket` as well as several several conditional operators that are options. More information on the conditional operations available can be found in the user manual at [Classical and Conditional Operations](https://tket.quantinuum.com/api-docs/classical.html).

**Contents**
* [Generating Wasm](#Generating-Wasm)
* [Basic Usage](#Basic-Usage)
* [Repetition Code](#Repetition-Code)
    * [TKET Classical Expressions](#TKET-Classical-Expressions)
    * [Wasm Hybrid Compute](#Wasm-Hybrid-Compute)
* [Summary](#Summary)

## Generating Wasm

## Basic Usage

## Repetition Code

### TKET Classical Expressions

Classical expressions can be applied to classical bits using the `pytket.circuit.logic_exp` submodule. These expressions perform boolean operations on classical bit registers.

A `pytket.circuit.Circuit` is constructed using the `QubitRegister` and `BitRegister` objects to allocate qubits and bits to the circuit. The circuit contains two quantum registers, `q` and `a`, and six classical registers:
* `c`
* `pfu`
* `pfu_new`
* `pfu_old`
* `syn`
* `syn_old`
* `syn_new`

In [9]:
from pytket.circuit import Circuit, QubitRegister, BitRegister

circuit = Circuit()

qreg = QubitRegister("q", 3)
circuit.add_q_register(qreg)

areg = QubitRegister("a", 1)
circuit.add_q_register(areg)

creg = BitRegister("c", 3)
circuit.add_c_register(creg)

pfu = BitRegister("pfu", 3)
circuit.add_c_register(pfu)

pfu_old = BitRegister("pfu_old", 3)
circuit.add_c_register(pfu_old)

syn = BitRegister("syn", 2)
circuit.add_c_register(syn)

syn_old = BitRegister("syn_old", 2)
circuit.add_c_register(syn_old)

syn_new = BitRegister("syn_new", 2)
circuit.add_c_register(syn_new)

BitRegister("syn_new", 2)

The code-cell below adds the necessary operations to measure the stabilizers $\hat{Z}_0 \hat{Z}_1$ and $\hat{Z}_1 \hat{Z}_2$. The ancilla qubit, `a[0]`, can be reused via the `OpType.Reset` operation.

In [ ]:
circuit.CX(qreg[0], areg[0])
circuit.CX(qreg[1], areg[0])
circuit.Measure(areg[0], creg[0])
circuit.Reset(areg[0])

circuit.CX(qreg[1], areg[0])
circuit.CX(qreg[2], areg[0])
circuit.Measure(areg[0], creg[1])
circuit.Reset(areg[0])

Classical operations on the syndrome bit registers are applied. A barrier is added to ensure these classical operations are applied after the syndrome extraction above. `pytket.circuit.logic_exp.RegXor` is used to perform the `XOR` operation on register `syn_old` and `syn`. This is added to the circuit with the instance method `add_classicalexpbox_register`. Finally the state of a classical register, `syn`, is copied to another classical_register, `syn_old`. `syn_old[0]`, `syn_old[1]` and `syn_old[2]` are equal to `syn[0]`, `syn[1]` and `syn[2]`.

In [ ]:
from pytket.circuit.logic_exp import RegXor

circuit.add_barrier(circuit.qubits + circuit.bits)
circuit.add_classicalexpbox_register(RegXor(syn_old, syn), syn_new)
circuit.add_c_copyreg(syn, syn_old)

The `pytket.circuit.Circuit` instance method, `add_c_setbits`, allows the end-user to set the value of specific bits on classical registers. The boolean argument `True` (`False`) corresponds to the bit value of `1` (`0`). This can also be conditioned on the value of specific bit by using the keyword argument `condition`. The function, `reg_eq`, is used to check equality of a bit to a specific value (10b is 2), and this output is passed to `condition` kwarg. 

In [ ]:
from pytket.circuit.logic_exp import reg_eq

circuit.add_c_setbits([True], [pfu[0]], condition=reg_eq(syn_new, 1))
circuit.add_c_setbits([True], [pfu[1]], condition=reg_eq(syn_new, 3))
circuit.add_c_setbits([True], [pfu[2]], condition=reg_eq(syn_new, 2))

`add_c_xor_to_registers` is used to set the value of `pfu_old` to the output of the expression, `pfu_old` ^ `pfu`. `add_c_setreg` is used to set the all the bits in the classical register, `pfu`, to `False`.

In [ ]:
circuit.add_c_xor_to_registers(pfu_old, pfu, pfu_old)
circuit.add_c_setreg(False, pfu)

Three measurement operations are applied to qubits within `qreg`. The outcome of the measurements is stored in the corresponding bits within `creg`, i.e. a measurement on `qreg[0]` will have a measurement outcome stored in `creg[0]`.

In [ ]:
circuit.add_barrier(circuit.qubits+circuit.bits)
for i in range(3):
    circuit.Measure(qreg[i], creg[i])

`add_classicalexpbox_bit`, `add_c_setbits`, `add_classicalexpbox_register` are used to perform additional operations on the classical registers. The `^` operation is also used instead of `RegXOR` to perform `XOR` on two classical registers. `add_classicalexpbox_bit` is used to perform operations on specific bits, whilst `add_classicalexpbox_register` allows classical operations on entire bit registers.

In [ ]:
circuit.add_barrier(circuit.qubits+circuit.bits)
circuit.add_classicalexpbox_bit(creg[0] ^ creg[1], [syn[0]])
circuit.add_classicalexpbox_bit(creg[1] ^ creg[2], [syn[1]])
circuit.add_classicalexpbox_register(syn_old ^ syn, syn_new)
circuit.add_c_setbits([True], [pfu[0]], condition=reg_eq(syn_new, 1))
circuit.add_c_setbits([True], [pfu[1]], condition=reg_eq(syn_new, 3))
circuit.add_c_setbits([True], [pfu[2]], condition=reg_eq(syn_new, 2))
circuit.add_classicalexpbox_register(pfu_old ^ pfu, pfu_old)
circuit.add_c_setreg(False, pfu)
circuit.add_classicalexpbox_register(pfu_old ^ creg, pfu_old)

### Wasm Hybrid Compute

The classical operations on bits and bit registers using `pytket.circuit.logic_exp` can be replaced with `Wasm`. The module `pytket.wasm`, in conjunction with `add_wasm_to_reg` and relevant keyword arguments in pytket-quantinuum, can be used to inject Wasm calls in the TKET program.

In [10]:
from pytket.circuit import Circuit, QubitRegister, BitRegister

circuit = Circuit()

qreg = QubitRegister("q", 3)
circuit.add_q_register(qreg)

areg = QubitRegister("a", 1)
circuit.add_q_register(areg)

creg = BitRegister("c", 3)
circuit.add_c_register(creg)

pfu = BitRegister("pfu", 3)
circuit.add_c_register(pfu)

pfu_old = BitRegister("pfu_old", 3)
circuit.add_c_register(pfu_old)

syn = BitRegister("syn", 2)
circuit.add_c_register(syn)

syn_old = BitRegister("syn_old", 2)
circuit.add_c_register(syn_old)

syn_new = BitRegister("syn_new", 2)
circuit.add_c_register(syn_new)

circuit.CX(qreg[0], areg[0])
circuit.CX(qreg[1], areg[0])
circuit.Measure(areg[0], creg[0])
circuit.Reset(areg[0])

circuit.CX(qreg[1], areg[0])
circuit.CX(qreg[2], areg[0])
circuit.Measure(areg[0], creg[1])
circuit.Reset(areg[0])

In [14]:
from pathlib import Path
from pytket.wasm import WasmFileHandler

wasm_file_path = Path().cwd().joinpath("repeat_until_success") / "rus.wasm"

wasm_file_handler = WasmFileHandler(wasm_file_path, check_file=True)

In [12]:
from pytket.circuit.logic_exp import RegXor

syndrome_list = []

for i in range(2):
    syndrome = BitRegister(f"syndrome-{i}", 2)
    circuit.add_c_register(syndrome)

    circuit.add_circbox(circ_box, [ancillas[0], qreg[0], qreg[1], syndrome[0]]) # zzi
    circuit.add_circbox(circ_box, [ancillas[0], qreg[1], qreg[2], syndrome[1]]) # izz
    circuit.add_wasm_to_reg("decode3", wasm_file_handler, [syndrome, syndrome_list[-1]])
    syndrome_list += [syndrome]

(syn ^ syn_old)
(syn ^ syn_old)


## Summary

In [5]:
circuit.add_wasm_to_reg?

Signature:
circuit.add_wasm_to_reg(
    funcname: str,
    filehandler: pytket.wasm.wasm.WasmFileHandler,
    list_i: Sequence[pytket._tket.unit_id.BitRegister],
    list_o: Sequence[pytket._tket.unit_id.BitRegister],
    args_wasm: Optional[Sequence[int]] = None,
    **kwargs: Any,
) -> pytket._tket.circuit.Circuit
Docstring:
Add a classical function call from a wasm file to the circuit.
    

:param funcname: name of the function that is called
    
:param filehandler: wasm file handler to identify the wasm file
    
:param list_i: list of the classical registers assigned to
     the input variables of the function call
    
:param list_o: list of the classical registers assigned to
     the output variables of the function call
    
:param args_wasm: vector of wasmstates the wasm op should be added to
    
:param kwargs: additional arguments passed to `add_gate_method` .
     Allowed parameters are `opgroup`,  `condition` , `condition_bits`,
     `condition_value`
    
:return: the 